In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import os
import json
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
Main_Languages = ['Haskell']
timeout_seconds = 4
timeout_seconds1 = 6

In [ ]:
problems = pd.read_csv('problems.csv',header=0)

In [ ]:
#f"/page/{Page_number}?order=BY_PROGRAM_LENGTH_ASC"

In [ ]:
def link_generator(problems):
  Links=[]
  for i, problem in problems.iterrows():
    contest_id = problem['contest_id']
    index = problem['index']
    Link_to_status = f"https://codeforces.com/problemset/status/{contest_id}/problem/{index}"
    Link_to_problem = f"https://codeforces.com/problemset/problem/{contest_id}/{index}"
    Links.append((Link_to_problem,Link_to_status))
  return Links
Links = link_generator(problems)

In [ ]:
Links[13]

In [ ]:
len(Links)

In [ ]:
def extract_links_ids_from_status_page(soup , LINK):
  """
  https://codeforces.com/problemset/status/1370/problem/D/page/1?order=BY_PROGRAM_LENGTH_ASC
  status page of one problem contain multible solutions
  I exctracted the solutions link and submissions id  that has language in MainLanguage list
  """
  table_of_submissions = soup.find('table', class_='status-frame-datatable')

  i=0
  Links_Ids = []

  if table_of_submissions == None:
    print(f"no table of submission in {LINK}")
    return Links_Ids
  try:
    #Access submission table
    for row in table_of_submissions.find_all('tr'):
      #Access cells in submission table
      cells = row.find_all('td')

      if len(cells) == 0:
        continue
      #Access Lang cell
      if len(cells) >= 5:
        language = cells[4].text.strip()
        if language not in Main_Languages:
          continue

      #Access "#" Cell
        first_cell = cells[0]
        a_tag = first_cell.find('a', class_='view-source')

        if a_tag:
          Link_to_source_solution = a_tag['href']
          submission_id = a_tag['submissionid']
          Links_Ids.append(("https://codeforces.com" + Link_to_source_solution , submission_id))
        else:
          print(f"Error2 - There is no a_Tag {LINK}\n")
      else:
        print(f"Error1 - There is no language value in this row {LINK}\n")
      i+=1
  except Exception as e:
      traceback.print_exc()
  return Links_Ids

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
"""URL = "https://codeforces.com/problemset/status/950/problem/B"
driver = web_driver()
driver.get(URL)

dropdown = driver.find_element(By.XPATH, "//select[@id='programTypeForInvoker']")
option = dropdown.find_element(By.XPATH, "//option[@value='cpp.g++17']")
option.click()

dropdown1 = driver.find_element(By.XPATH, "//select[@id='verdictName']")
option1 = dropdown1.find_element(By.XPATH, "//option[@value='WRONG_ANSWER']")
option1.click()

button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Apply']")  # Locate the button using XPath
button.click()
soup = BeautifulSoup(driver.page_source, "html.parser")
"""

In [ ]:

#URL = "https://codeforces.com/contest/4/status/A"

#page = requests.get(URL)

def extract_submissions_links_from_all_status( URL):
  try:
    driver = web_driver()
    driver.get(URL)

    dropdown = driver.find_element(By.XPATH, "//select[@id='programTypeForInvoker']")
    option = dropdown.find_element(By.XPATH, "//option[@value='haskell.ghc']")
    option.click()

    dropdown1 = driver.find_element(By.XPATH, "//select[@id='verdictName']")
    option1 = dropdown1.find_element(By.XPATH, "//option[@value='OK']")
    option1.click()

    button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Apply']")  # Locate the button using XPath
    button.click()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    extracted_data=[]
    try:
      extracted_data.extend(extract_links_ids_from_status_page(soup, URL))
    except:
      print(f"Error - in exctracting submissions_links_from_all_status: {URL}")
  except Exception as e:
    print(f"Error - in driver {URL} ")
    traceback.print_exc()
  return extracted_data
  """pagination_div = soup.find_all('div', class_="pagination")[-1]
  last_page_number = -1
  if pagination_div:
    ul_element = pagination_div.find('ul')
    if ul_element:
      last_page_number = ul_element.find_all('li')[-2].text.strip()
    else:
      print("Error - no <ul>\n")
      print(link)
  else:
    print("Error - no <li>\n")

  print(last_page_number,f" last page number for {link}\n")

  if last_page_number == -1:
    last_page_number = 10

  for page_number in range(1,int(last_page_number)+1):
    URL_pages = link +  f"/page/{page_number}?order=BY_PROGRAM_LENGTH_ASC"
    try:
      other_page = requests.get(URL_pages , timeout=timeout_seconds)
      if other_page.status_code != 200:
        print(f"Error - could not access to status page num: {URL_pages}")
        continue
      other_soup = BeautifulSoup(other_page.content, "html.parser")
      extracted_data.extend(extract_links_ids_from_status_page(other_soup, URL_pages))
      #print(len(extracted_data) , page_number)
    except:
      print(f"Error - could not access to status page num: {URL_pages}")
      pass
  return extracted_data
  """
#ex = extract_submissions_links_from_all_status(URL)
#ex

In [ ]:
#Link1= "https://codeforces.com/problemset/submission/1370/84700462"
#page = requests.get(Link1)
#print(page.status_code)
#soup = BeautifulSoup(page.content, 'html.parser')
def extract_code_from_submission(soup , LINK):
  try:
    code = soup.find('pre', id='program-source-text').text.strip()
    return code
  except:
    print(f"Error submission did not find code {LINK}\n")
    return ""
#code = extract_code_from_submission(soup, Link1)

In [ ]:
#print(code)

In [ ]:
from numpy.core.multiarray import empty
from bs4.element import NavigableString
def extract_text_mathlb_from_problem(position):
  data = ""
  math_pos = []
  try:
    if len(position.contents) > 1:
      for child in position.children:
          X , math_pos1 = extract_text_mathlb_from_problem(child)
          if child.name == "div" or child.name == "p":
              data += "\n"
              math_pos.extend([0,0])
          else:
              data += " "
              math_pos.extend([0])
          data += X
          math_pos.extend(math_pos1)
    else:
      if position.name == 'span' and position.get('class') == ['MathJax']:
          data = position.get('data-mathml')

          math_pos.extend([1 for i in range(len(data))])
      else:
          data = position.get_text(strip=False)
          math_pos.extend([0 for i in range(len(data))])
  except:
    if str(position).strip():
      data = str(position).strip()
      math_pos.extend([0 for i in range(len(data))])
  return data , math_pos

In [ ]:
Link1= "https://codeforces.com/contest/1370/problem/F1"
page = requests.get(Link1)
soup = BeautifulSoup(page.content, 'html.parser')
problem_statement = soup.find(class_='problem-statement')
Data , pos = extract_text_mathlb_from_problem(problem_statement)

In [ ]:
print(Data)

In [ ]:
len(Links)

In [ ]:
DataSet = []

In [ ]:
import concurrent.futures
import requests
import traceback
from bs4 import BeautifulSoup
# Specify the timeout value in seconds
import threading



j = 0
problems_num1 = 0
problems_num2 = 0
status_num1 = 0
submissions_num1 = 0
lock = threading.Lock()
lock1 = threading.Lock()
lock2 = threading.Lock()
lock3 = threading.Lock()
def process_link(link):
    problem_link = link[0]
    status_link = link[1]
    try:
      lock.acquire()
      try:
        page = requests.get(problem_link ,timeout=1)
      finally:
        lock.release()
      global problems_num1
      global  problems_num2
      global submissions_num1
      global status_num1
      global j
      global DataSet
      problems_num1 += 1
      if page.status_code != 200:
        print(f"Error - could not access to problem link: {problem_link}","\n")
        problems_num2 += 1
        return

      print(f"Problem {problems_num1} , {problems_num2} , {problem_link}", page.status_code,"\n")
      soup = BeautifulSoup(page.content, 'html.parser')
      problem_statement = soup.find(class_='problem-statement')
      problem_data, pos = extract_text_mathlb_from_problem(problem_statement)
      try:
        lock1.acquire()
        try:
          submissions_links = extract_submissions_links_from_all_status(status_link)
        finally:
         lock1.release()
        # Critical section: perform operations while the lock is held
        print(status_link , submissions_links,"\n")
        print(problem_link,len(submissions_links),"\n")
        for submission in submissions_links:
          try:
            page = requests.get(submission[0] ,timeout=timeout_seconds)
            if page.status_code != 200:
              print(f"Error - could not access to submission link: {status_link}\n")
              submissions_num1 += 1
              continue
            print(f"Submission {submission[0]} , f{submissions_num1}",page.status_code,"\n")
            soup = BeautifulSoup(page.content, 'html.parser')
            code = extract_code_from_submission(soup, submission[0])
            if code == " ":
              submissions_num1+=1
              continue
            DATA_FORM = {
                'problem_statement': problem_data,
                'solution': code,
                'solution_link': submission[0],
                'solution_id': submission[0],
                'problem_link': problem_link
            }
            lock3.acquire()
            try:
              DataSet.append(DATA_FORM)
            finally:
              lock3.release()
            j += 1
            print(j, " number of submissions till now\n")
          except Exception as e:
            traceback.print_exc()
            print(f"could not access to the submission {submission[0]}\n")
      except Exception as e:
        traceback.print_exc()
        print(f"could not access to the status_link {status_link}\n")
    except Exception as e:
      traceback.print_exc()
      print(f"could not access to the problem link {problem_link}\n")

def make_json(Links):
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        executor.map(process_link, Links)

make_json(Links[1000:2000])

In [ ]:
"""key_to_check = "solution"

unique_values = set()

for item in DataSet:
    print(item["solution_id"],item["problem_link"])
    value = item[key_to_check]
    if value is not None:
        unique_values.add(value)
print(len(unique_values))
"""

In [ ]:
import csv

# Specify the output file path
output_file = 'output1to2.csv'

# Extract the keys from the first dictionary in the list
keys = DataSet[0].keys()

# Open the file in write mode
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=keys)

    # Write the header row
    writer.writeheader()

    # Write each dictionary as a CSV row
    writer.writerows(DataSet)

print(f'CSV file "{output_file}" created successfully.')

In [ ]:
!pip install -U -q PyDrive


In [ ]:
!zip codeforces-v2-1to2.zip /content/output1to2.csv

In [ ]:
!cp /content/codeforces-v2-1to2.zip /content/drive/MyDrive/codeforces-v2-1to2.zip